## W2V Dash

### Supporting Functions

In [ ]:
def tree_parser(node, inner_list, outer_list, root_node, depth):
    d = node.key_to_child_node  # dict
    for token in list(d.keys()):
        if len(root_node.key_to_child_node.keys()) == 0:
            ret_list = []
            for row in outer_list:
                proper_len = int(row[1])
                if len(row) == proper_len+1 or len(row) + 1 == depth:
                    ret_list.append(row)
            return ret_list
        inner_list.append(token)
        child = d[token]
        if child.key_to_child_node:
            tree_parser(child, inner_list, outer_list, root_node, depth)
        else:
            d.pop(token)
            outer_list.append(inner_list)
            inner_list = ['root']
            tree_parser(root_node, inner_list, outer_list, root_node, depth)

In [ ]:
def tree_to_list_parser(node):
    tree_df = []
    curr_path = []
    tree_dict = {}
    prev_root = [("root", node)]
    while len(prev_root) > 0:
        # Peek at last value
        curr_root = prev_root[-1]

        # Get the node element
        curr_node = curr_root[1].key_to_child_node

        # Follow path value if not already there
        if len(curr_path) <= 0 or curr_path[-1] != curr_root[0]:
            curr_path.append(curr_root[0])

        visited = False
        if curr_root[1] in tree_dict:
            visited = True
        else:
            tree_dict[curr_root[1]] = True

        # Check if value has any leaf nodes
        if not visited and len(curr_node.keys()) > 0:
            # Add those to the stack
            for nn in curr_node.items():
                prev_root.append((nn[0], nn[1]))
        else:
            # Remove previous node in the path
            prev_root.pop()

            # Record to the database if leaf
            if len(curr_node.keys()) <= 0:
                tree_df.append(deepcopy(curr_path))

            # Move back up tree
            curr_path.pop()
    return tree_df

In [ ]:
def appendSpherical_np(xyz):
    ptsnew = np.hstack((xyz, np.zeros(xyz.shape)))
    xy = xyz[:, 0]**2 + xyz[:, 1]**2
    ptsnew[:, 3] = np.sqrt(xy + xyz[:, 2]**2)
    ptsnew[:, 4] = np.arctan2(np.sqrt(xy), xyz[:, 2])  # for elevation angle defined from Z-axis down
    # ptsnew[:,4] = np.arctan2(xyz[:,2], np.sqrt(xy)) # for elevation angle defined from XY-plane up
    ptsnew[:, 5] = np.arctan2(xyz[:, 1], xyz[:, 0])
    return ptsnew

In [ ]:
def get_spherical_coords(xyz):
    sph = np.zeros(shape=xyz.shape)
    xy = xyz[:, 0]**2 + xyz[:, 1]**2
    sph[:, 0] = np.sqrt(xy + xyz[:, 2]**2)
    sph[:, 1] = np.arctan2(np.sqrt(xy), xyz[:, 2])
    sph[:, 2] = np.arctan2(xyz[:, 1], xyz[:, 0])
    return sph

The output of the W2V pipeline is a matrix of size [vocab size x embedding size]

### Environmental Variables

In [ ]:
# -- W2V Dash Environmental Variables -- #

W2V_NEIGHBORS = 20
RECURSION_LIMIT = 10**6
N_PROJ_DIM = 3
DASH_SEED = 0

### Generate Projection Data

In [ ]:
# -- Generate Data for Word Embeddings Projector -- #

# shape = vocab size x embedding dim size
weights = np.ndarray(shape=(len(embed_weights), w2v_config["embed_size"]))

# -- Populate Matrix for PCA -- #
for idx, weight in enumerate(list(embed_weights.values())):
    weights[idx, :] = weight

# -- Dimensionality Reduction -- #
pca = PCA(n_components=N_PROJ_DIM, random_state=DASH_SEED).fit(weights)
ica = FastICA(n_components=N_PROJ_DIM, random_state=DASH_SEED).fit(weights)
srp = SparseRandomProjection(n_components=N_PROJ_DIM, random_state=DASH_SEED).fit(weights)
reduced_embeddings = pca.transform(weights)

# -- Calculate Nearest Neighbors -- #
model = NearestNeighbors(n_neighbors=W2V_NEIGHBORS, algorithm='auto')
trained_embeddings = model.fit(reduced_embeddings)

# Currently the array has a shape of vocab size x N_PROJ_DIM and contains
# the fitted PCA data. We need to add the vocab in the first column so
# we know which vectors are represented.
scatter_plot_3d_cols = ['token', 'x1', 'x2', 'x3']
embedding_vocab_arr = np.array(list(embed_weights.keys()))
embedding_vocab_arr = np.expand_dims(embedding_vocab_arr, 1)
named_reduced_embeddings = np.hstack((embedding_vocab_arr, reduced_embeddings))
scatter_plot_3d_df = pd.DataFrame(
    data=named_reduced_embeddings,
    columns=scatter_plot_3d_cols)
scatter_plot_3d_df['x1'] = pd.to_numeric(scatter_plot_3d_df['x1'])
scatter_plot_3d_df['x2'] = pd.to_numeric(scatter_plot_3d_df['x2'])
scatter_plot_3d_df['x3'] = pd.to_numeric(scatter_plot_3d_df['x3'])

We will build our plot using the tree_parser function. This function recursively
steps through the drain3.TemplateMiner.drain.Node structure of our
**TextClusteringLayer** (TCL). The recursion populates a np.array which is then used
to build a pandas dataframe which the plotly treemap accepts. There is a column
appended to the tail of the dataframe which counts the number of stars
(wild card masks) present in the row. This is used to define the colors shown.

### Generate Treemap Data

In [ ]:
# By default python's recursion limit is 10**4 which is too small for our needs
sys.setrecursionlimit(RECURSION_LIMIT)

# The root node is the master node of the tree and will be our return point
root_node = deepcopy(w2vp.TCL.template_miner.drain.root_node)
parsed_tree = tree_to_list_parser(root_node)
parsed_tree_df = pd.DataFrame(data=parsed_tree)

# The returned dataframe has generic columns so we will provide custom labels
n_cols = len(parsed_tree_df.columns)
col_name_list = []
for idx in range(n_cols):
    col_name_list.append('level' + str(idx))
parsed_tree_df.columns = col_name_list

'''
Without a color column our treemap would just be plain. We thought that taking
the sum of the drain mask would be an interesting way to color the treemap.
This lambda function will sum those values in each row and return them to a new
columnn named 'sum'
'''
parsed_tree_df['sum'] = parsed_tree_df.apply(lambda x: x.str.contains('<*>'), axis=1).sum(axis=1)  # noqa

### Dash Variables

In [ ]:
pio.templates.default = "plotly_dark"
external_stylesheets_url = 'https://drive.google.com/uc?export=view&id=19OXGQ5iJIjRZD4VEZ-xiVChDmj0-SlSF'  # noqa
external_stylesheets = [external_stylesheets_url]

CACHE_CONFIG = dict()
CACHE_CONFIG['CACHE_TYPE'] = 'filesystem'
CACHE_CONFIG['CACHE_DIR'] = SOURCE + '/results/dash_cache'

### Colors

In [ ]:
color_d = dict()
color_d['blue'] = 'rgb(66, 133, 244)'
color_d['red'] = 'rgb(219, 68, 55)'
color_d['yellow'] = 'rgb(244, 180, 0)'
color_d['orange'] = 'rgb(255, 165, 0)'
color_d['green'] = 'rgb(15, 157, 88)'
color_d['mint'] = 'rgb(3, 218, 198)'
color_d['dark mint'] = 'rgb(1, 135, 134)'
color_d['dark purple'] = 'rgb(55, 0, 179)'
color_d['purple'] = 'rgb(98, 0, 238)'

### Dash Formatting

In [ ]:
# ================= #
#  3d Scatter Plot  #
# ================= #

# Line formatting
scatter_plot_3d_line = dict()
scatter_plot_3d_line['width'] = 2
scatter_plot_3d_line['color'] = color_d['dark mint']

scatter_plot_3d_selected_line = dict()
scatter_plot_3d_selected_line['width'] = 2
scatter_plot_3d_selected_line['color'] = color_d['dark mint']

scatter_plot_3d_nonselected_line = dict()
scatter_plot_3d_nonselected_line['width'] = 2
scatter_plot_3d_nonselected_line['color'] = color_d['dark mint']

scatter_plot_3d_darker_line = dict()
scatter_plot_3d_darker_line['width'] = 2
scatter_plot_3d_darker_line['color'] = color_d['dark purple']


# Marker formatting
scatter_plot_3d_marker = dict()
scatter_plot_3d_marker['size'] = 5
scatter_plot_3d_marker['line'] = scatter_plot_3d_line
scatter_plot_3d_marker['color'] = color_d['mint']

scatter_plot_3d_selected_marker = dict()
scatter_plot_3d_selected_marker['size'] = 5
scatter_plot_3d_selected_marker['color'] = color_d['mint']
scatter_plot_3d_selected_marker['line'] = scatter_plot_3d_selected_line

scatter_plot_3d_nonselected_marker = dict()
scatter_plot_3d_nonselected_marker['size'] = 5
scatter_plot_3d_nonselected_marker['color'] = color_d['mint']
scatter_plot_3d_nonselected_marker['opacity'] = 0.15
scatter_plot_3d_nonselected_marker['line'] = scatter_plot_3d_nonselected_line

scatter_plot_3d_marker_no_color = dict()
scatter_plot_3d_marker_no_color['size'] = 5
scatter_plot_3d_marker_no_color['line'] = scatter_plot_3d_darker_line

scatter_plot_3d_marker_cluster_center = dict()
scatter_plot_3d_marker_cluster_center['size'] = 10
scatter_plot_3d_marker_cluster_center['color'] = color_d['orange']
scatter_plot_3d_marker_cluster_center['opacity'] = 0.5
scatter_plot_3d_marker_cluster_center['line'] = scatter_plot_3d_darker_line

scatter_plot_3d_selected_table_marker = dict()
scatter_plot_3d_selected_table_marker['size'] = 5
scatter_plot_3d_selected_table_marker['color'] = color_d['yellow']
scatter_plot_3d_selected_table_marker['line'] = scatter_plot_3d_darker_line


# Style
scatter_plot_3d_style = dict()
scatter_plot_3d_style['height'] = '100%'
scatter_plot_3d_style['width'] = '100%'


# ========= #
#  Treemap  #
# ========= #

# Style
treemap_style = dict()
treemap_style['height'] = '100%'
treemap_style['width'] = '100%'


# ============ #
#  Data Table  #
# ============ #

# Style
data_table_cell_style = dict()
data_table_cell_style['textAlign'] = 'left'
data_table_cell_style['overflow'] = 'hidden'
data_table_cell_style['textOverflow'] = 'ellipsis'
data_table_cell_style['maxWidth'] = 0
data_table_cell_style['backgroundColor'] = 'rgb(20, 20, 20)'
data_table_cell_style['color'] = 'white'

data_table_header_style = dict()
data_table_header_style['backgroundColor'] = color_d['purple']


# ======== #
#  Labels  #
# ======== #

# Style
clustering_alg_drop_down_label_style = dict()
clustering_alg_drop_down_label_style['color'] = 'white'

coordinate_space_drop_down_label_style = dict()
coordinate_space_drop_down_label_style['color'] = 'white'

dim_reduction_drop_down_label_style = dict()
dim_reduction_drop_down_label_style['color'] = 'white'

### Dash Configuration

In [ ]:
# ================= #
#  3d Scatter Plot  #
# ================= #
scatter_plot_3d_config = dict()
scatter_plot_3d_config['responsive'] = True


# ========= #
#  Treemap  #
# ========= #
treemap_config = dict()
treemap_config['responsive'] = True

### Dash Dropdown Options

In [ ]:
clustering_alg_drop_down_options = [
    {'label': 'KNN', 'value': 'KNN'},
    {'label': 'GMM', 'value': 'GMM'},
    {'label': 'Bayesian GMM', 'value': 'BGMM'},
    {'label': 'Affinity Prop.', 'value': 'AP'},
    {'label': 'KMEANS', 'value': 'KM'},
    {'label': 'SVM', 'value': 'SVM'},
]

coordinate_space_drop_down_options = [
    {'label': 'Cartesian', 'value': 'CT'},
    {'label': 'Spherical', 'value': 'SP'}
]

dim_reduction_drop_down_options = [
    {'label': 'PCA', 'value': 'PCA'},
    {'label': 'ICA', 'value': 'ICA'},
    {'label': 'LDA', 'value': 'LDA'},
    {'label': 'Sparse RP', 'value': 'SRP'},
    {'label': 'Gaussian RP', 'value': 'GRP'}
]

### Dash Main

In [ ]:
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
cache = Cache()
cache.init_app(app.server, config=CACHE_CONFIG)


# =============== #
#  Cluster Table  #
# =============== #
table = pd.DataFrame(
    data=list(embed_weights.keys()),
    columns=['token'])

# ============= #
#  Scatterplot  #
# ============= #
scatter_plot_3d_fig = px.scatter_3d(
                      scatter_plot_3d_df,
                      x='x1',
                      y='x2',
                      z='x3',
                      hover_name='token')

scatter_plot_2d_fig = px.scatter(
                     scatter_plot_3d_df,
                     x='x1',
                     y='x2',
                     hover_name='token')

scatter_plot_3d_fig.update_traces(marker=scatter_plot_3d_marker)
scatter_plot_3d_fig['layout']['uirevision'] = 1

scatter_plot_2d_fig.update_traces(marker=scatter_plot_3d_marker)
scatter_plot_2d_fig['layout']['uirevision'] = 1


# ========= #
#  Treemap  #
# ========= #
treemap_fig = px.treemap(
    parsed_tree_df,
    path=col_name_list,
    color='sum')


# ============ #
#  App Layout  #
# ============ #
app.layout = html.Div([

        html.Div([

            # -- Clustering Technique Dropdown -- #
            html.Label(
                "Clustering Algorithm (TODO)",
                style=clustering_alg_drop_down_label_style),
            dcc.Dropdown(
                id='cluster-dropdown',
                options=clustering_alg_drop_down_options,
                value='KNN'),

            # -- Coordinate Space Dropdown -- #
            html.Label(
                "Coordinate Space",
                style=coordinate_space_drop_down_label_style),
            dcc.Dropdown(
                id='coord-dropdown',
                options=coordinate_space_drop_down_options,
                value='CT'),

            # -- Dimensionality Reduction Technique Dropdown -- #
            html.Label(
                "Dimensionality Reduction (TODO)",
                style=dim_reduction_drop_down_label_style),
            dcc.Dropdown(
                id='dr-dropdown',
                options=dim_reduction_drop_down_options,
                value='PCA'
            )
        ], className='options-graph-container'),

        # -- 3d Scatter Plot -- #
        html.Div(
            [dcc.Graph(
                id='3d_scat',
                figure=scatter_plot_3d_fig,
                config=scatter_plot_3d_config,
                style=scatter_plot_3d_style),
             dcc.Slider(
                id='my-slider',
                min=0.5,
                max=0.9,
                step=0.05,
                value=0.5)],
            className='main-graph-container',
            id='graph_div'),

        # -- Tree Map -- #
        html.Div(
            dcc.Graph(
                id='3d_tree',
                figure=treemap_fig,
                config=treemap_config,
                style=treemap_style),
            className='secondary-graph-container',
            id='tree_div'),

        # -- Neighbors Datatable -- #
        html.Div(
            children=[dash_table.DataTable(
                 id='table',
                 columns=[{"name": i, "id": i} for i in table.columns],
                 data=pd.DataFrame().to_dict('records'),
                 style_cell=data_table_cell_style,
                 style_header=data_table_header_style,
             )],
            className='related-graph',
            id='data_table'),

        # signal value to trigger callbacks
        dcc.Store(id='signal')],

    id='report-container')


# ============= #
#  Memoization  #
# ============= #

# Table of Contents:
# -----------------------------
# 1. Projection DataFrame
# 2. Coordinates
# 3. Dimensionality Reductions
# 4. Clustering Algorithms
# -----------------------------

# -- 1. Projection DataFrame -- #
@cache.memoize()
def dataframe_store(embeddings):
    new_df = pd.DataFrame(
        data=embeddings,
        columns=scatter_plot_3d_cols)
    new_df['x1'] = pd.to_numeric(new_df['x1'])
    new_df['x2'] = pd.to_numeric(new_df['x2'])
    new_df['x3'] = pd.to_numeric(new_df['x3'])
    return new_df


# -- 2. Coordinates -- #
@cache.memoize()
def coordinate_space_store(value, embeddings):
    # calculate new coordinate space
    if value == 'SP':
        spherical_embeddings = get_spherical_coords(embeddings)
        embeddings_stack_tup = (embedding_vocab_arr, spherical_embeddings)
        named_embeddings = np.hstack(embeddings_stack_tup)
    elif value == "CT":
        embeddings_stack_tup = (embedding_vocab_arr, embeddings)
        named_embeddings = np.hstack(embeddings_stack_tup)
    else:
        return no_update
    return named_embeddings


# -- 3. Dimensionality Reduction -- #
@cache.memoize()
def dimension_reduct_store(value):
    # calculate new dimensionality reduction algorithm
    if value == "PCA":
        dr_embeddings = pca.transform(weights)
    elif value == "ICA":
        dr_embeddings = ica.transform(weights)
    elif value == "SRP":
        dr_embeddings = srp.transform(weights)
    else:
        return no_update
    return dr_embeddings


# -- 4. Clustering Algorithms -- #
@cache.memoize()
def clustering_algo_store(value, damp_value):
    # calculate new clustering algorithm
    if value == "KNN":
        model = NearestNeighbors(n_neighbors=W2V_NEIGHBORS, algorithm='auto')
    elif value == "AP":
        model = AffinityPropagation(damping=damp_value, random_state=DASH_SEED)
    elif value == "KM":
        model = KMeans(n_clusters=4)
    elif value == "GMM":
        model = GaussianMixture(n_components=4)
    elif value == "SVM":
        model = SVC(kernel='poly', degree=3, probability=True, random_state=DASH_SEED)
    return model


# =========== #
#  Callbacks  #
# =========== #

# -- Calculate Projection Data -- #
@app.callback(Output('signal', 'data'),
              Input('dr-dropdown', 'value'),
              Input('cluster-dropdown', 'value'),
              Input('coord-dropdown', 'value'),
              Input('my-slider', 'value'))
def compute_coordinate_space(dr_val, cluster_val, coord_val, damp_value):
    return (dr_val, cluster_val, coord_val, damp_value)


# -- Point Selection Mechanics -- #
@app.callback(Output("table", "data"),
              Output("3d_scat", "figure"),
              Input('3d_scat', 'clickData'),
              Input("signal", "data"),
              Input("table", "selected_rows"))
def select_point(clickData, value, rows):
    ctx = dash.callback_context
    ids = [c['prop_id'] for c in ctx.triggered]

    embeddings = dimension_reduct_store(value[0])
    model = clustering_algo_store(value[1], value[3])
    named_embeddings = coordinate_space_store(value[2], embeddings)
    df = dataframe_store(named_embeddings)

    clustering_model = model.fit(named_embeddings[:, 1:4].astype(float))

    if '3d_scat.clickData' in ids:
        if clickData:
            for p in clickData['points']:
                if value[1] != "KNN":
                    return no_update, no_update

                coord_list = [p['x'], p['y'], p['z']]
                query_arr = np.array(coord_list).reshape(1, -1)

                _, neighbors = clustering_model.kneighbors(X=query_arr)
                neighbors_list = neighbors.tolist()[0]
                tokens = []
                for idx in neighbors_list:
                    tokens.append(table.iloc[idx])
                update = pd.DataFrame(data=tokens)

                selected_df = df[df.index.isin(neighbors_list)]
                nonselected_df = df.drop(index=neighbors_list)

                ff = px.scatter_3d(
                    selected_df,
                    x='x1',
                    y='x2',
                    z='x3',
                    hover_name='token')

                ff = ff.update_traces(marker=scatter_plot_3d_selected_marker)

                if rows is not None:
                    table_point = selected_df[selected_df['token'] == rows]
                    ff2_1 = px.scatter_3d(
                            table_point,
                            x='x1',
                            y='x2',
                            z='x3',
                            text='token')

                    ff2_1 = ff2_1.update_traces(marker=scatter_plot_3d_selected_table_marker)
                    ff.add_trace(ff2_1.data[0])

                ff2 = px.scatter_3d(
                    nonselected_df,
                    x='x1',
                    y='x2',
                    z='x3',
                    hover_name='token')

                ff2 = ff2.update_traces(marker=scatter_plot_3d_nonselected_marker)

                ff.add_trace(ff2.data[0])
                ff['layout']['uirevision'] = 1

                return update.to_dict('records'), ff
    elif 'signal.data' in ids:
        if value[1] != "KNN":
            y_pred = clustering_model.predict(embeddings)

            df.insert(0, "Label", y_pred, True)
            ff = px.scatter_3d(
                df,
                x='x1',
                y='x2',
                z='x3',
                color='Label',
                hover_name='token')

            ff.update_traces(marker=scatter_plot_3d_marker_no_color)

            if "GMM" not in value[1]:
                centers = pd.DataFrame(data=clustering_model.cluster_centers_, columns=["x1", "x2", "x3"])
                ff2 = px.scatter_3d(
                    centers,
                    x='x1',
                    y='x2',
                    z='x3')
                ff2.update_traces(marker=scatter_plot_3d_marker_cluster_center)

                ff.add_trace(ff2.data[0])
        else:
            ff = px.scatter_3d(
                df,
                x='x1',
                y='x2',
                z='x3',
                hover_name='token')

            ff.update_traces(marker=scatter_plot_3d_marker)

        ff['layout']['uirevision'] = 1

        return no_update, ff
    else:
        return no_update, no_update


app.run_server(host='0.0.0.0', mode='jupyterlab')


